# 数据规整话方面的话题
## 时间序列以及截面对齐

In [ ]:
import pandas as pd
import numpy as np
from datetime import time

In [ ]:
! head ../pydata-book-master/ch11/volume.csv

In [ ]:
# 成交价格
close_px=pd.read_csv(filepath_or_buffer='../pydata-book-master/ch11/stock_px.csv',
                     parse_dates=True,index_col=0)
volume=pd.read_csv(filepath_or_buffer='../pydata-book-master/ch11/volume.csv',
                     parse_dates=True,index_col=0)
prices = close_px.loc['2011-09-05':'2011-09-14', ['AAPL', 'JNJ', 'SPX', 'XOM']]
volume = volume.loc['2011-09-05':'2011-09-12', ['AAPL', 'JNJ', 'XOM']]
prices

In [ ]:
# 成交量
volume

In [ ]:
# pandas dataframe d.mul()
prices.mul(other=volume)

In [ ]:
# 计算各个公司股票的加权平均价格
# pandas series s.div()
vwap = prices.mul(other=volume).sum().div(volume.sum())
vwap

In [ ]:
# pandas series s.dropna()
# 显式丢弃SPX的缺失值
vwap.dropna()

In [ ]:
# 手工对齐
# pandas dataframe d.align() join
prices.align(other=volume,join='inner')

In [ ]:
# python Built-in Functions range()
s1 = pd.Series(range(3),index=['a','b','c'])
s2 = pd.Series(range(4),index=['d','b','c','e'])
s3 = pd.Series(range(3),index=['f','a','c'])
pd.DataFrame({'one':s1,'two':s2,'three':s3})

In [ ]:
# python Built-in Functions list()
pd.DataFrame({'one':s1,'two':s2,'three':s3},index=list('face'))

## 频率不同的时间序列的运算

In [ ]:
ts1 = pd.Series(np.random.randn(3),
                index=pd.date_range(start='2012-6-13',periods=3,freq='W-WED'))
ts1

In [ ]:
# pandas series s.resample()
# pandas Resampling r.mean()
ts1.resample(rule='B').mean()

In [ ]:
# pandas Resampling r.mean()
ts1.resample(rule='B').ffill()

In [ ]:
# pandas Index pd.DatetimeIndex()
dates = pd.DatetimeIndex(['2012-6-12','2012-6-17','2012-6-18','2012-6-21','2012-6-22','2012-6-29'])
ts2 = pd.Series(np.random.randn(6),index=dates)
ts2

In [ ]:
# pandas series s.reindex()
ts1.reindex(ts2.index,method='ffill')

In [ ]:
# pandas series s.add()
ts2.add(ts1.reindex(ts2.index,method='ffill'))

## 使用Period
pandas General funciton pd.period_range() periods

In [ ]:
gdp = pd.Series([1.78,1.94,2.08,2.01,2.15,2.31,2.46],
               index=pd.period_range(start='1984Q2',periods=7,freq='Q-SEP'))
infl = pd.Series([0.025,0.045,0.037,0.04],
                index=pd.period_range(start='1982',periods=4,freq='A-DEC'))
gdp

In [ ]:
infl

In [ ]:
# pandas series s.asfreq() how
infl_q = infl.asfreq(freq='Q-SEP',how='end')
infl_q

In [ ]:
infl_q.reindex(gdp.index,method='ffill')

## 时间和“最当前”数据选取
pandas index i.append()  
python Built-in Functions range()  
pandas Date offsets tseries.offsets.BusinessDay() BDay  
python List Comprehensions 列表推导  
numpy Array creation routines np.arange() dtype  
python Built-in Functions float()

In [ ]:
# 生成一个交易日内的日期范围和时间索引
rng = pd.date_range(start='2012-06-01 09:30',end='2012-06-01 15:59',freq='T')
# 生成5天的时间点（9：30-15：59之间的值）
rng = rng.append([rng + pd.offsets.BDay(i) for i in range(1,4)])
ts = pd.Series(np.arange(len(rng),dtype=float),index=rng)
ts.head()

In [ ]:
# pandas Indexing and Selecting Data series[label]
# python datetime time()
ts[time(hour=10,minute=0)]

In [ ]:
# python datetime time()
# pandas series s.at_time()
ts.at_time(time(hour=10,minute=0))

In [ ]:
# pandas series s.between_time()
ts.between_time(time(hour=10,minute=0),time(hour=10,minute=1))

In [ ]:
# 可能刚好就没有任何数据落在某个具体的时间上，这时您可能会希望得到上午10点之前最后出现的哪个值
# 将该时间序列的大部分内容随机设置为NA
# numpy Random sampling (numpy.random) np.random.permutation()
# numpy Sorting, searching, and counting np.sort()
# pandas series s.copy()
indexer = np.sort(np.random.permutation(len(ts))[700:])
irr_ts = ts.copy()
irr_ts[indexer] = np.nan
irr_ts['2012-06-01 09:50':'2012-06-01 10:00']

In [ ]:
# pandas General functions pd.date_range()
# pandas series s.asof()
selection = pd.date_range(start='2012-06-01 10:00',periods=4,freq='B')
irr_ts.asof(where=selection)

## 拼接多个数据源
经常出现的情况
+ 在一个特定的时间点上，从一个数据源切换到另一个数据源
+ 用另一个时间序列对当前时间序列中的缺失值打补丁
+ 将数据中的符号（国家、资产代码等）替换为实际数据

python Built-in Functions float()  
numpy Array creation routines np.ones() dtype  
numpy Broadcasting 标量  
pandas General functions pd.concat()  

In [ ]:
data1 = pd.DataFrame(np.ones((6,3),dtype=float),
                    columns=['a','b','c'],
                    index=pd.date_range(start='2012/6/12',periods=6))
data2 = pd.DataFrame(np.ones((6,3),dtype=float)*2,
                    columns=['a','b','c'],
                    index=pd.date_range(start='2012/6/13',periods=6))
spliced = pd.concat([data1.loc[:'2012-06-14'],data2.loc['2012-06-15':]])
spliced

In [ ]:
data2 = pd.DataFrame(np.ones((6,4),dtype=float)*2,
                    columns=['a','b','c','d'],
                    index=pd.date_range(start='2012/6/13',periods=6))
spliced = pd.concat([data1.loc[:'2012-06-14'],data2.loc['2012-06-15':]],sort=False)
spliced

In [ ]:
# combine_first可以引入合并点之前的数据，这样也就扩展了'd'项的历史
# 由于data2没有关于2012-06-12的数据，所以也就没有值被填充到那一天
# pandas series s.combine_first()
spliced_filled=spliced.combine_first(data2)
spliced_filled

# 分组变化和分析